In [3]:
from pathlib import Path

import polars as pl

DATE = "2023-12-01"
REPORT_DIR = Path("../reports/scores")
PORTFOLIO_DIR = Path("../reports/portfolios")

In [9]:
for portfolio in PORTFOLIO_DIR.glob("portfolio_*.xlsx"):
    df = pl.read_excel(portfolio, sheet_name="Full Portfolio")
    top = df.head(5)

    if "fwd_return_4Q" not in df.columns:
        continue

    top_freturn = top.select(pl.col("fwd_return_4Q")).mean().item()
    top_hits = top.select(pl.col("fwd_return_4Q") > 0).sum().item()

    top_hitrate = (top_hits / len(top)) * 100

    print(f"\nPortfolio: {portfolio}")
    print(f"Average return: {top_freturn:.2f}% ({top_hitrate:.1f})")


Portfolio: ../reports/portfolios/portfolio_2023-06-01.xlsx
Average return: 32.73% (80.0)

Portfolio: ../reports/portfolios/portfolio_2023-03-01.xlsx
Average return: 59.51% (100.0)

Portfolio: ../reports/portfolios/portfolio_2023-12-01.xlsx
Average return: 21.08% (80.0)
